In [7]:
from nltk.corpus import gutenberg
import time
macbeth = gutenberg.raw('austen-persuasion.txt')
fileids = gutenberg.fileids()
print(fileids)
data = macbeth.replace('\n', '')
import sys
from collections import defaultdict

def MR(data):
    def filter(str):
        str = str.lower()
        i = [x for x in range(97)] 
        j = [x for x in range(123,256)]
        k = i + j
        for c in [chr(i) for i in k]:
            str = str.replace(c, '')
        return str
    
    def lzw_encode(data):
        # Build the dictionary.
        # lowercase!!
        data = filter(data)
        global initial_size_MR 
        initial_size_MR = len(data)
        max_dict_size = 256
        dictionary = {chr(i): i for i in range(max_dict_size)}
        hadict = [defaultdict(int) for _ in range(26)]

        dict_size = [0]*26
        p = ""
        result = []
        for i in range(26):
            hadict[i][chr(ord('a') + i)] = 0

        for c in data:
            pc = p + c
            if pc in hadict[ord(c)- ord('a')]:
                p = pc
            else:
                if(p == ""): result.append("#")
                elif hadict[ord(p[-1]) - ord('a')][p] == 0 : result.append('#'+p[-1])
                else: result.append(str(hadict[ord(p[-1]) - ord('a')][p]) + p[-1])
                # if dict_size < 4096:  # Typically the dictionary size is limited to 4096 entries.
                val = ord(c)- ord('a')
                hadict[val][pc] = len(hadict[val])
                p = c

        if p:
            val = ord(c)- ord('a')
            result.append(str(hadict[val][p]) + p[-1])
        return "".join(result)
    return lzw_encode(data)
def lzw_encode(data):
    # Build the dictionary.
    def filter(str):
        str = str.lower()
        i = [x for x in range(97)] 
        j = [x for x in range(123,256)]
        k = i + j
        for c in [chr(i) for i in k]:
            str = str.replace(c, '')
        return str
    data = filter(data)
    global initial_size_lzw 
    initial_size_lzw = len(data)
    max_dict_size = 26
    dictionary = {chr(i): i for i in range(97,123)}
    dict_size = max_dict_size
    p = ""
    result = []

    for c in data:
        pc = p + c
        if pc in dictionary:
            p = pc
        else:
            if len(p) == 1: result.append('#'+p)
            else: result.append(str(dictionary[p]))
            # if dict_size < 4096:  # Typically the dictionary size is limited to 4096 entries.
            dictionary[pc] = str(dict_size)
            dict_size += 1
            p = c

    if p:
        result.append(str(dictionary[p]))

    return "".join(result)
def lzw_decode(compressed):
    # Build the dictionary.
    max_dict_size = 256
    dictionary = {i: chr(i) for i in range(max_dict_size)}
    dict_size = max_dict_size

    # Decoding first value
    result = []
    w = chr(compressed.pop(0))
    result.append(w)

    for k in compressed:
        if k in dictionary:
            entry = dictionary[k]
        elif k == dict_size:
            entry = w + w[0]
        else:
            raise ValueError("Bad compressed k: %s" % k)
        
        result.append(entry)

        # Add w+entry[0] to the dictionary.
        if dict_size < 4096:
            dictionary[dict_size] = w + entry[0]
            dict_size += 1

        w = entry

    return "".join(result)

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [12]:

# Example usage
# decompressed = lzw_decode(compressed)
# print("Decompressed:", decompressed)
for fileid in fileids:
    data = gutenberg.raw(fileid)
    print('FileId:', fileid)
    start_time_MR = time.time()
    encoded_MR = MR(data)
    end_time_MR = time.time()
    total_time_MR = end_time_MR - start_time_MR
    final_size_MR = len(encoded_MR)
    start_time_lzw = time.time()
    encoded_lzw = lzw_encode(data)
    end_time_lzw = time.time()
    total_time_lzw = end_time_lzw - start_time_lzw
    final_size_lzw = len(encoded_lzw)
    print('Compression Rate MR: ', (initial_size_MR-final_size_MR)/initial_size_MR * 100)
    print('Time Taken MR:', total_time_MR)
    print('Compression Rate LZW: ', (initial_size_lzw-final_size_lzw)/initial_size_lzw * 100)
    print('Time Taken LZW:', total_time_lzw)
    print('Percentage increase in time taken by MR over LZW:', (total_time_MR-total_time_lzw)/total_time_lzw * 100)
    print()


FileId: austen-emma.txt
Compression Rate MR:  14.806956323512004
Time Taken MR: 0.2868218421936035
Compression Rate LZW:  10.114316693627462
Time Taken LZW: 0.19913411140441895
Percentage increase in time taken by MR over LZW: 44.034510295978706
FileId: austen-persuasion.txt
Compression Rate MR:  12.001194857726976
Time Taken MR: 0.14578676223754883
Compression Rate LZW:  6.588982644417466
Time Taken LZW: 0.10491275787353516
Percentage increase in time taken by MR over LZW: 38.959994182294174
FileId: austen-sense.txt
Compression Rate MR:  13.85830910119228
Time Taken MR: 0.20822405815124512
Compression Rate LZW:  9.326851116582782
Time Taken LZW: 0.15266895294189453
Percentage increase in time taken by MR over LZW: 36.38926195458663
FileId: bible-kjv.txt
Compression Rate MR:  22.26869658235633
Time Taken MR: 1.3555760383605957
Compression Rate LZW:  18.06910095684594
Time Taken LZW: 1.0200271606445312
Percentage increase in time taken by MR over LZW: 32.89607283633889
FileId: blake-poe